# 실습: Q&A 애플리케이션

LangChian과 LangGraph 설치

In [ ]:
!pip install langchain==0.3.0 langchain-openai==0.2.0 langgraph==0.2.22

OpenAI API 키 설정

In [ ]:
from google.colab import userdata
import os

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['LANGCHAIN_TRACING_V2'] = userdata.get('LANGSMITH_TRACING')
os.environ['LANGCHAIN_ENDPOINT'] = userdata.get('LANGSMITH_ENDPOINT')
os.environ['LANGCHAIN_API_KEY'] = userdata.get('LANGSMITH_API_KEY')
os.environ['LANGCHAIN_PROJECT'] = userdata.get('LANGSMITH_PROJECT')

역할 정의

In [ ]:
ROLES = {
    "1": {
        "name": "일반 지식 전문가",
        "description": "폭넓은 분야의 일반적인 질문에 답변",
        "details": "폭넓은 분야의 일반적인 질문에 대해 정확하고 이해하기 쉬운 답변을 제공하세요."
    },
    "2": {
        "name": "생성형 AI 제품 전문가",
        "description": "생성형 AI와 관련 제품, 기술에 관한 전문적인 질문에 답변",
        "details": "생성형 AI와 관련 제품, 기술에 관한 전문적인 질문에 대해 최신 정보와 깊은 통찰력을 제공하세요."
    },
    "3": {
        "name": "카운슬러",
        "description": "개인적인 고민이나 심리적인 문제에 대해 지원 제공",
        "details": "개인적인 고민이나 심리적인 문제에 대해 공감적이고 지원적인 답변을 제공하고, 가능하다면 적절한 조언도 해주세요."
    }
}

스테이트 정의

In [ ]:
import operator
from typing import Annotated
from langchain_core.pydantic_v1 import BaseModel, Field

class State(BaseModel):
  query: str = Field(..., description="사용자의 질문")
  current_role: str = Field(default="", description="선정된 답변 역할")
  messages: Annotated[list[str], operator.add] = Field(default=[],description="답변 이력")
  current_judge: bool = Field(default=False, description="품질 체크 결과")
  judgement_reason: str = Field(default="", description="품질 체크 판정 이유")

Chat model 초기화

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.runnables import ConfigurableField

llm = ChatOpenAI(model="gpt-4o", temperature=0.0)
# 나중에 max_tokens 값을 변경할 수 있도록 변경 가능한 필드 선언
llm = llm.configurable_fields(max_tokens=ConfigurableField(id='max_tokens'))

노드 정의

In [ ]:
from typing import Any
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

def selection_node(state: State) -> dict[str, Any]:
  query = state.query
  role_options = "\n".join([f"{k}. {v['name']}: {v['description']}" for k, v in ROLES.items()])

  prompt = ChatPromptTemplate.from_template("""
  질문을 분석하고, 가장 적절한 답변 담당 역할을 선택하세요.
  선택지:
  {role_options}

  답변은 선택지의 번호{{1, 2, 또는 3}}만 반환하세요.

  질문: {query}
  """.strip())

  # 선택지의 번호만 반환하기를 기대하므로 max_tokens 값을 1로 변경
  chain = prompt | llm.with_config(configurable=dict(max_tokens=1)) | StrOutputParser()
  role_number = chain.invoke({"role_options": role_options, "query": query})
  selected_role = ROLES[role_number.strip()]["name"]

  return {"current_role": selected_role}

answering 노드 구현

In [ ]:
def answering_node(state: State) -> dict[str, Any]:
  query = state.query
  role = state.current_role
  role_details = "\n".join([f"- {v['name']}: {v['details']}" for v in ROLES.values()])
  prompt = ChatPromptTemplate.from_template("""
  당신은 {role}로서 답변하세요. 다음 질문에 대해 당신의 역할에 기반한 적잘한 답변을 제공하세요.

  역할 상세:
  {role_details}

  질문: {query}

  답변:""".strip()
  )
  chain = prompt | llm | StrOutputParser()
  answer = chain.invoke({"role": role, "role_details": role_details, "query": query})
  return {"messages": [answer]}

check 노드 구현

In [ ]:
class Judgement(BaseModel):
  judge: bool = Field(default=False, description="판정 결과")
  reason: str = Field(default="", description="판정 이유")

def check_node(state: State) -> dict[str, Any]:
  query = state.query
  answer = state.messages[-1]
  prompt = ChatPromptTemplate.from_template("""
  다음 답변의 품질을 체크하고, 문제가 있으면 'False', 문제가 없으면 'True'로 답변하세요.
  또한, 그 판정 이유도 설명하세요.

  사용자의 질문: {query}
  답변: {answer}
  """.strip()
  )
  chain = prompt | llm.with_structured_output(Judgement)
  result: Judgement = chain.invoke({"query": query, "answer": answer})

  return {
      "current_judge": result.judge,
      "judgement_reason": result.reason
  }

그래프 생성

In [ ]:
from langgraph.graph import StateGraph

workflow = StateGraph(State)

노드 추가

In [ ]:
workflow.add_node("selection", selection_node)
workflow.add_node("answering", answering_node)
workflow.add_node("check",check_node)

에지 정의

In [ ]:
# selection 노드에서 처리 시작
workflow.set_entry_point("selection")

# selection 노드에서 answering 노드로
workflow.add_edge("selection", "answering")

# answering 노드에서 check 노드로
workflow.add_edge("answering", "check")

조건부 에지 정의

In [ ]:
from langgraph.graph import END

# check 노드에서 다음 노드로의 전환에 조건부 에지 정의
# state.current_judge 값이 True면 END 노드로, False면 selection 노드로
workflow.add_conditional_edges(
    "check",
    lambda state: state.current_judge,
    {True: END, False: "selection"}
)

그래프 컴파일

In [ ]:
compiled = workflow.compile()

그래프 실행

In [ ]:
initial_state = State(query="생성형 AI에 관해 알려주세요")
result = compiled.invoke(initial_state)

결과 표시

In [ ]:
print(result["messages"][-1])

그래프 구조 시각화 표시

In [ ]:
!apt-get install graphviz libgraphviz-dev pkg-config
!pip install pygraphviz

In [ ]:
from IPython.display import Image

Image(compiled.get_graph().draw_png())